Vamos a pasar el .pkl a .csv para polars

In [24]:
import polars as pl
import pandas as pd

train = pl.read_csv("../../data/raw/train.csv")
clients = pl.read_csv("../../data/raw/users_data.csv")
products_pandas = pd.read_pickle("../../data/raw/products.pkl")
products = pl.from_pandas(products_pandas)

In [7]:
products

discount,embedding,partnumber,color_id,cod_section,family
"decimal[1,0]",list[f32],i32,i32,f64,i32
0,"[-0.134014, -0.120043, … -0.080103]",32776,85,4.0,73
0,"[-0.094927, -0.107294, … -0.044575]",41431,135,4.0,73
0,"[-0.129044, -0.077246, … 0.002047]",39419,339,4.0,73
1,"[-0.127833, -0.133868, … -0.091194]",36087,135,4.0,73
1,"[-0.140929, -0.125828, … -0.043056]",34132,3,4.0,73
…,…,…,…,…,…
0,"[-0.172583, 0.780391, … -0.110433]",234,368,2.0,192
0,"[-0.081145, 0.717163, … -0.06411]",235,152,2.0,138
0,"[-0.151772, 1.640085, … -0.100086]",236,226,2.0,135


In [20]:
result_q1 = (
    products
    .filter((pl.col("color_id") == 3) & (pl.col("discount") == 1))
    .sort("family")
    .select("partnumber")
    .head(1)
)

print("Q1 Result:", result_q1)


Q1 Result: shape: (1, 1)
┌────────────┐
│ partnumber │
│ ---        │
│ i32        │
╞════════════╡
│ 17265      │
└────────────┘


In [40]:
# Filtrar usuarios con compras totales < 500 y contar cuántos hay en cada país
filtered_clients = (
    clients
    .filter(pl.col("M") < 500)  # Filtrar por compras < 500
    .group_by("country")  # Agrupar por país
    .agg(pl.len().alias("count"))  # Contar los usuarios por país
    .sort("count", descending=True)  # Ordenar por cantidad de usuarios
    .select('country')
    .head(1)
)

# País con más usuarios con compras < 500
top_country = filtered_clients["country"]

# Filtrar usuarios en el país top y ordenar por F, R, y user_id
result_q2 = (
    clients
    .filter(pl.col("country") == top_country)  # Filtrar por el país top
    .sort(["F", "R", "user_id"], descending=[False, True, False])  # Ordenar
    .select('user_id')  # Obtener el primer usuario
    .head(1)
)

print("Q2 Result:", result_q2)

Q2 Result: shape: (1, 1)
┌─────────┐
│ user_id │
│ ---     │
│ i64     │
╞═════════╡
│ 187374  │
└─────────┘


In [41]:
train

session_id,date,timestamp_local,add_to_cart,user_id,country,partnumber,device_type,pagetype
i64,str,str,i64,f64,i64,i64,i64,f64
64,"""2024-06-06""","""2024-06-06 16:43:17.389""",0,null,29,14327,1,24.0
117,"""2024-06-08""","""2024-06-08 15:11:02.782""",0,null,57,38422,1,24.0
117,"""2024-06-08""","""2024-06-08 15:11:44.797""",0,null,57,19763,1,24.0
579,"""2024-06-05""","""2024-06-05 19:24:48.397""",0,null,29,30253,1,24.0
1220,"""2024-06-04""","""2024-06-04 08:21:13.476""",0,480729.0,25,1592,1,24.0
…,…,…,…,…,…,…,…,…
5170695,"""2024-06-07""","""2024-06-07 17:57:24.644""",0,null,34,39901,3,24.0
5171109,"""2024-06-04""","""2024-06-04 12:34:05.430""",0,null,29,38638,1,24.0
5171307,"""2024-06-07""","""2024-06-07 07:50:02.549""",0,null,25,10883,1,24.0


In [53]:
# result_q3 = (
#     train
#     .filter(pl.col("add_to_cart") == 1)  # Filtrar productos añadidos al carrito
#     .group_by("partnumber")
#     .agg([
#         pl.len().alias("total_visits"),
#         pl.sum("add_to_cart").alias("cart_adds"),
#     ])
#     #.with_columns((pl.col("total_visits") / pl.col("cart_adds")).alias("visits_per_cart_add"))
#     .select(pl.col("visits_per_cart_add").mean().round(2))
# )

result_q3_filtered= (
    train
    .group_by('partnumber')  # Agrupar por 'partnumber'
    .agg(pl.sum('add_to_cart').alias('cart_adds'))  # Sumar 'add_to_cart' para obtener 'cart_adds'
    .filter(pl.col('cart_adds') > 1)  # Filtrar solo los productos con 'cart_adds' > 1
)

result_q3 = (
    train
    .filter(pl.col('partnumber').is_in(result_q3_filtered['partnumber']))
    .group_by("partnumber")
    .agg([
        pl.len().alias("total_visits"),
        pl.sum("add_to_cart").alias("cart_adds"),
    ])
    .with_columns((pl.col("total_visits") / pl.col("cart_adds")).alias("visits_per_cart_add"))
    .select(pl.col("visits_per_cart_add").mean().round(2))
)

print(result_q3)



shape: (1, 1)
┌─────────────────────┐
│ visits_per_cart_add │
│ ---                 │
│ f64                 │
╞═════════════════════╡
│ 21.9                │
└─────────────────────┘


In [ ]:
result_q4 = (
    train
    .filter((pl.col("add_to_cart") == 1) & (pl.col("discount") == 1))
    .groupby("device_type")
    .count()
    .sort("count", reverse=True)
    .select("device_type")
    .first()
)

print("Q4 Result:", result_q4)


In [ ]:
top_users_by_frequency = (
    clients
    .groupby("country")
    .agg(pl.col("purchase_frequency").top_k(3).alias("top_frequencies"))
    .explode("top_frequencies")
)

result_q5 = (
    train
    .filter(pl.col("device_type") == 3)
    .join(top_users_by_frequency, on="user_id", how="inner")
    .groupby("user_id")
    .agg(pl.n_unique("partnumber").alias("unique_interactions"))
    .sort("unique_interactions", reverse=True)
    .first()
)

print("Q5 Result:", result_q5)


In [ ]:
result_q6 = (
    train
    .filter(pl.col("country") != pl.col("user_country"))  # Filtrar interacciones fuera del país del usuario
    .select("family_identifier")
    .n_unique()
)

print("Q6 Result:", result_q6)


In [ ]:
from datetime import datetime

# Filtrar interacciones de los primeros 7 días de junio
result_q7 = (
    train
    .filter(
        (pl.col("add_to_cart") == 1) &
        (pl.col("date").str.strptime(pl.Date, fmt="%Y-%m-%d") >= datetime(2023, 6, 1)) &
        (pl.col("date").str.strptime(pl.Date, fmt="%Y-%m-%d") <= datetime(2023, 6, 7))
    )
    .groupby(["family", "page_type"])
    .count()
    .sort(["family", "count", "page_type"], descending=[False, True, False])
    .groupby("family")
    .agg(pl.first("page_type").alias("most_frequent_pagetype"))
    .to_dicts()
)

print("Q7 Result:", result_q7)